# preprocess LOWZ target photometry and spectroscopy

In [1]:
import os
import h5py
import numpy as np
import astropy.table as aTable

In [2]:
from provabgs import desi as DESI

In [3]:
# -- plotting -- 
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rcParams['font.family'] = 'serif'
mpl.rcParams['axes.linewidth'] = 1.5
mpl.rcParams['axes.xmargin'] = 1
mpl.rcParams['xtick.labelsize'] = 'x-large'
mpl.rcParams['xtick.major.size'] = 5
mpl.rcParams['xtick.major.width'] = 1.5
mpl.rcParams['ytick.labelsize'] = 'x-large'
mpl.rcParams['ytick.major.size'] = 5
mpl.rcParams['ytick.major.width'] = 1.5
mpl.rcParams['legend.frameon'] = False


Bad key text.latex.preview in file /global/homes/c/chahah/.conda/envs/gqp/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle, line 123 ('text.latex.preview : False')
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.5.1/matplotlibrc.template
or from the matplotlib source distribution

Bad key mathtext.fallback_to_cm in file /global/homes/c/chahah/.conda/envs/gqp/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle, line 155 ('mathtext.fallback_to_cm : True  # When True, use symbols from the Computer Modern')
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.5.1/matplotlibrc.template
or from the matplotlib source distribution

Bad key savefig.jpeg_quality in file /global/homes/c/chahah/.conda/envs/gqp/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle, line 418 ('savefig.jpeg_quality: 95 

In [4]:
dat_dir = '/global/cfs/cdirs/desi/users/chahah/provabgs/lowz'

In [5]:
lowz_targ = aTable.Table.read(os.path.join(dat_dir, 'provabgs_lowz_targets.fits'))

In [6]:
print('%i lowz targets' % len(lowz_targ))

121277 lowz targets


In [7]:
def find_lowztarget_coadd(healpix, targetid): 
    # find healpix coadd file with LOWZ target
    
    redux = ['fuji', 'guadalupe']
    survey = ['sv1', 'sv3', 'main']
    
    for r in redux: 
        for s in survey: 
            for b in ['dark', 'bright']: 
                fcoadd = os.path.join('/global/cfs/cdirs/desi/spectro/redux/', 
                                      '%s/healpix/%s/%s/%s/%i/coadd-%s-%s-%i.fits' % (r, s, b, str(healpix)[:-2], healpix, s, b, healpix))
                if os.path.isfile(fcoadd):
                    coadd = aTable.Table.read(fcoadd)
    
                    if np.sum(coadd['TARGETID'] == targetid): 
                        return fcoadd
                              
    raise ValueError("couldn't find target") 

In [ ]:
for ii in range(7, 25): 
    print('%i of 25' % ii)
    
    targid = []
    f_photo, i_photo = [], [] 
    w_spec, f_spec, i_spec = [], [], [] 
    f_fib, sig_f_fib = [], [] 
    redshifts = [] 

    for i in range(5000 * ii, np.min([5000 * (ii+1), len(lowz_targ)])): 
        if i % 100 == 0: print('\t %i' % i)
        fcoadd = find_lowztarget_coadd(lowz_targ['HEALPIX'][i], lowz_targ['TARGETID'][i])
        frrock = fcoadd.replace('coadd', 'redrock')

        coadd = aTable.Table.read(fcoadd)
        rrock = aTable.Table.read(frrock)
        
        is_target = (coadd['TARGETID'] == lowz_targ['TARGETID'][i])

        spec = DESI.readDESIspec(fcoadd)

        # spectra
        w_obs = np.concatenate([spec['wave_b'], spec['wave_r'], spec['wave_z']])
        f_obs = np.concatenate([spec['flux_b'], spec['flux_r'], spec['flux_z']],
                axis=1)
        i_obs = np.concatenate([spec['ivar_b'], spec['ivar_r'], spec['ivar_z']],
                axis=1)

        # sort the wavelength
        isort = np.argsort(w_obs)
        w_obs = w_obs[isort]
        f_obs = f_obs[is_target,:][:,isort]
        i_obs = i_obs[is_target,:][:,isort]

        # photometry (de-redden)
        coadd = coadd[is_target]
        # assign photsys
        photsys = 'S'
        if coadd['TARGET_DEC'] > 32.375: photsys = 'N' 
        
        trans_g = DESI.mwdust_transmission(coadd['EBV'], 'g',
                np.array([photsys]).astype(str),
                match_legacy_surveys=False)
        trans_r = DESI.mwdust_transmission(coadd['EBV'], 'r',
                np.array([photsys]).astype(str),
                match_legacy_surveys=False)
        trans_z = DESI.mwdust_transmission(coadd['EBV'], 'z',
                np.array([photsys]).astype(str),
                match_legacy_surveys=False)

        flux_g = coadd['FLUX_G'] / trans_g
        flux_r = coadd['FLUX_R'] / trans_r
        flux_z = coadd['FLUX_Z'] / trans_z
        fiberflux_r = coadd['FIBERFLUX_R'] / trans_r

        #list(coadd['FLUX_G', 'FLUX_R', 'FLUX_Z'].as_array())).copy()
        photo_flux = np.array([flux_g, flux_r, flux_z]).T
        photo_ivar = np.array(list(coadd['FLUX_IVAR_G', 'FLUX_IVAR_R', 'FLUX_IVAR_Z'].as_array())).copy()

        # fiber flux fraction estimate
        f_fiber = coadd['FIBERFLUX_R'] / coadd['FLUX_R']
        sigma_f_fiber = f_fiber * coadd['FLUX_IVAR_R']**-0.5

        # redshift
        zred = rrock['Z'][is_target]

        f_photo.append(photo_flux)
        i_photo.append(photo_ivar)

        w_spec.append(w_obs)
        f_spec.append(f_obs)
        i_spec.append(i_obs)

        f_fib.append(f_fiber)
        sig_f_fib.append(sigma_f_fiber)
        
        redshifts.append(zred)
        targid.append(lowz_targ['TARGETID'][i])
    
    fh5 = h5py.File(os.path.join(dat_dir, 'lowz.obs.%iof25.hdf5' % ii), 'w')
    fh5.create_dataset('TARGETID', data=np.array(targid).astype(int))
                       
    fh5.create_dataset('photo_flux', data=np.array(f_photo))
    fh5.create_dataset('photo_ivar', data=np.array(i_photo))
    
    fh5.create_dataset('spec_wave', data=np.array(w_spec))
    fh5.create_dataset('spec_flux', data=np.array(f_spec))
    fh5.create_dataset('spec_ivar', data=np.array(i_spec))
    
    fh5.create_dataset('fiber_flux', data=np.array(f_fib))
    fh5.create_dataset('fiber_sigma_flux', data=np.array(sig_f_fib))
    
    fh5.create_dataset('redshift', data=np.array(redshifts))
    fh5.close()

7 of 25
	 35000


	 35100
	 35200
	 35300
	 35400
	 35500
	 35600
	 35700
	 35800
	 35900
	 36000
	 36100
	 36200
	 36300
	 36400
	 36500
	 36600
	 36700
	 36800
	 36900
	 37000
	 37100
	 37200
	 37300
	 37400
	 37500
	 37600
	 37700
	 37800
	 37900
	 38000
	 38100
	 38200
	 38300
	 38400
	 38500
	 38600
	 38700
	 38800
	 38900
	 39000
	 39100
	 39200
	 39300
	 39400
	 39500
	 39600
	 39700
	 39800
	 39900
8 of 25
	 40000
	 40100
	 40200
	 40300
	 40400
	 40500
	 40600
	 40700
	 40800
	 40900
	 41000
	 41100
	 41200
	 41300
	 41400
	 41500
	 41600
	 41700
	 41800
	 41900
	 42000
	 42100
	 42200
	 42300
	 42400
	 42500
	 42600
	 42700
	 42800
	 42900
	 43000
	 43100
	 43200
	 43300
	 43400
	 43500
	 43600
	 43700
	 43800
	 43900
	 44000
	 44100
	 44200
	 44300
	 44400
	 44500
	 44600
	 44700
	 44800
	 44900
9 of 25
	 45000
	 45100
	 45200
	 45300
	 45400
	 45500
	 45600
	 45700
	 45800
	 45900
	 46000
	 46100
	 46200
	 46300
	 46400
	 46500
	 46600
	 46700
	 46800
	 46900
	 47000
	 47100
	 47200
	 47300
